In [ ]:
from kreis_loader import get_envelope, get_kreise
from stations_loader import stations_find, filter_stations
from data_handler import SQLite
import json

# Open the file for reading
with open('../../kreise.json', 'r') as json_file:
    kreise = json.load(json_file)

for kreis in kreise:
    kreis["attributes"]['KREISID'] = kreis["attributes"].pop('OBJECTID')

# kreis table
columns = {
        'KREISID': 'INTEGER PRIMARY KEY NOT NULL',
        'ags': 'TEXT',
        'gen': 'TEXT',
        'bez': 'TEXT',
        'ibz': 'INTEGER',
        'bem': 'TEXT',
        'sn_l': 'TEXT',
        'sn_r': 'TEXT',
        'sn_k': 'TEXT',
        'sn_v1': 'TEXT',
        'sn_v2': 'TEXT',
        'sn_g': 'TEXT',
        'fk_s3': 'TEXT',
        'nuts': 'TEXT',
        'wsk': 'TEXT', 
        'ewz': 'INTEGER',
        'kfl': 'REAL',
        'Shape__Area': 'REAL',
        'Shape__Length': 'REAL'
    }
with SQLite('ChargeApp.db') as db:
        db.create_table("kreis_table", columns)

# geometry table
columns = {
            'KREISID': 'INTEGER PRIMARY KEY NOT NULL',
            'GeoData': 'BLOB',
        }   
reference_key = {
        'table': 'kreis_table',
        'column': 'KREISID',
        'reference_column': 'KREISID'
    }
with SQLite('ChargeApp.db') as db:
    db.create_sub_table("geometry", columns, reference_key)

# station table
columns = {
    'OBJECTID': 'INTEGER PRIMARY KEY NOT NULL',
    'KREISID': 'INTEGER',
    'Betreiber': 'TEXT',
    'Straße': 'TEXT',
    'Hausnummer': 'TEXT',
    'Adresszusatz': 'TEXT',
    'Postleitzahl': 'INTEGER',
    'Ort': 'TEXT',
    'Bundesland': 'TEXT',
    'Kreis_kreisfreie_Stadt': 'TEXT',
    'Breitengrad': 'REAL',
    'Längengrad': 'REAL',
    'Inbetriebnahmedatum': 'TEXT',
    'Anschlussleistung': 'REAL',
    'Art_der_Ladeeinrichung': 'TEXT',
    'Anzahl_Ladepunkte': 'INTEGER',
    'Steckertypen1': 'TEXT',
    'P1__kW_': 'INTEGER',
    'Public_Key1': 'TEXT',
    'Steckertypen2': 'TEXT',
    'P2__kW_': 'REAL',
    'Public_Key2': 'TEXT',
    'Steckertypen3': 'TEXT',
    'P3__kW_': 'INTEGER',
    'Public_Key3': 'TEXT',
    'Steckertypen4': 'TEXT',
    'P4__kW_': 'INTEGER',
    'Public_Key4': 'TEXT'
}
reference_key = {
        'table': 'kreis_table',
        'column': 'KREISID',
        'reference_column': 'KREISID'
    }
with SQLite('ChargeApp.db') as db:
    db.create_sub_table("stations", columns, reference_key)
    

In [ ]:
from kreis_loader import get_envelope, get_kreise
from stations_loader import stations_find, filter_stations
from data_handler import SQLite
import json

for kreis in kreise:
    kreis_id = kreis["attributes"].get('KREISID')

    ## Add Kreise
    with SQLite('ChargeApp.db') as db:
        db.insert_data(
            table_name="kreis_table",
            key_column="KREISID",
            data=kreis["attributes"]
        )

    polygon = kreis.get("geometry", None)

    ## Add geometry
    if polygon is not None:
        with SQLite('ChargeApp.db') as db:
            db.insert_data(
                table_name="geometry",
                key_column="KREISID",
                data={'GeoData': json.dumps(polygon)},
                key_value=kreis_id,
                reference_key={
                    'table': 'kreis_table',
                    'column': 'KREISID',
                    'reference_column': 'KREISID'
                }
            )

        # Compute the envelope and add it to the kreis
        envelope = get_envelope(polygon)
        with SQLite('ChargeApp.db') as db:
            db.add_column("kreis_table", "envelope", "TEXT")
            db.insert_data(
                table_name="kreis_table",
                key_column="KREISID",
                data={"envelope": kreis["envelope"]},
                key_value=kreis_id
            )

    ## Add stations
    stations = stations_find(geometry = envelope)
    stations_filtered = filter_stations(polygon, stations)
    for station in stations_filtered:
        station['KREISID'] = kreis_id

    with SQLite('ChargeApp.db') as db:
        db.insert_data(
            table_name="stations",
            key_column="OBJECTID",
            data=stations_filtered,
            reference_key={
                'table': 'kreis_table',
                'column': 'OBJECTID',
                'reference_column': 'KREISID'
            },
            strict=False
        )